In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import json
import open3d as o3d

from dust3r.inference import inference_with_mask
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.cloud_opt.base_opt import global_alignment_loop
from masked_dust3r.scripts.utils.math import *
from masked_dust3r.scripts.utils.image import *


DATA_PATH = "/dust3r/masked_dust3r/data/jackal_training_data_0"
IMG_FILE_EXTENSION = ".png"
MASK_FILE_EXTENSION = ".png"
GAUSSIAN_SIGMA = 51.0 #TODO: Dynamically size GAUSSIAN_SIGMA based on mask size?
INIT_FRAMES = 50
NEW_FRAMES = 10
PREVIOUS_FRAMES = 40
TOTAL_FRAMES = 300

INIT_WEIGHT_FOCAL = 0.0
INIT_WEIGHT_Z = 0.0001 
INIT_WEIGHT_ROT = 0.0001 * 0
INIT_WEIGHT_TRANS_SMOOTHNESS = 0.0001 * 0
INIT_WEIGHT_ROT_SMOOTHNESS = 0.001 * 0

NEW_WEIGHT_FOCAL = 0.1
NEW_WEIGHT_Z = 0.1
NEW_WEIGHT_ROT = 0.1
NEW_WEIGHT_TRANS_SMOOTHNESS = 0.00001
NEW_WEIGHT_ROT_SMOOTHNESS = 0.00001

USE_COMMON_INTRINSICS = True
IS_FOCAL_FIXED = True
FOCAL_LENGTH = 4.74

RESCALE_FACTOR = 1280/512 #TODO: Dynamically size RESCALE_FACTOR based on image size?

device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


In [3]:
# Load the model

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth


instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [4]:
images_array = []
masks_array = []

for i in range(0,250,2):
    images_array.append(os.path.join(DATA_PATH,"masked_images/{}{}".format(i,IMG_FILE_EXTENSION)))
    masks_array.append(os.path.join(DATA_PATH,"masks/{}{}".format(i,MASK_FILE_EXTENSION)))
images = load_images(images_array, size=512, verbose=True)
_,_,H,W = images[0]["img"].shape
masks = load_masks(masks_array, H, W, device)

>> Loading a list of 125 images
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/0.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/2.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/4.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/6.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/8.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/10.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/12.png with resolution 1280x720 --> 512x288


 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/14.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/16.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/18.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/20.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/22.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/24.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/26.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/28.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_dat

In [5]:
pairs = make_pairs(images, scene_graph='swin-2', prefilter=None, symmetrize=True)
output = inference_with_mask(pairs, model, device, masks, GAUSSIAN_SIGMA, batch_size=batch_size)
del pairs

>> Inference with model on 500 image pairs


  0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 500/500 [02:35<00:00,  3.21it/s]


In [6]:
#init_scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer)
#loss = init_scene.compute_global_alignment(init="mst", niter=niter, schedule='cosine', lr=lr)

#scene = init_scene

scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer, 
                        weight_focal = INIT_WEIGHT_FOCAL,
                        weight_z = INIT_WEIGHT_Z ,
                        weight_rot = INIT_WEIGHT_ROT  ,
                        weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS,
                        weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS)
#scene.im_poses = calculate_new_params(init_scene.im_poses,device)
#scene.im_focals = init_scene.im_focals
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#averge_focal = scene.get_focals().sum().item()/len(images_array)
#fixed_focal = [averge_focal for _ in range(len(images_array))]
#mask = [True for _ in range(len(images_array))]
#scene.preset_focal(fixed_focal, mask)
#loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#scene.weight_focal = INIT_WEIGHT_FOCAL
#scene.weight_z = INIT_WEIGHT_Z 
#scene.weight_rot = INIT_WEIGHT_ROT
#scene.weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS 
#scene.weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS
#loss = global_alignment_loop(scene, niter=niter, schedule=schedule, lr=lr)


 init edge (97*,96*) score=1.49197256565094
 init edge (96,98*) score=1.323627233505249
 init edge (97,99*) score=1.323294758796692
 init edge (97,95*) score=1.2902644872665405
 init edge (99,101*) score=1.1929433345794678
 init edge (96,94*) score=1.1316139698028564
 init edge (92*,94) score=1.1220128536224365
 init edge (90*,92) score=1.366672396659851
 init edge (91*,90) score=1.3349539041519165
 init edge (103*,101) score=1.3254626989364624
 init edge (103,102*) score=1.2662492990493774
 init edge (90,88*) score=1.2555385828018188
 init edge (91,93*) score=1.207076907157898
 init edge (103,105*) score=1.2029998302459717
 init edge (104*,102) score=1.1939443349838257
 init edge (104,106*) score=1.1668717861175537
 init edge (108*,106) score=1.1634293794631958
 init edge (102,100*) score=1.1542226076126099
 init edge (108,107*) score=1.1068669557571411
 init edge (90,89*) score=1.3925464153289795
 init edge (108,110*) score=1.294345498085022
 init edge (110,111*) score=1.276914715766

100%|██████████| 300/300 [06:10<00:00,  1.23s/it, lr=1.27413e-06 loss=0.000103888]


In [7]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()
intrinsics = scene.get_intrinsics()

In [8]:
#Check if pointclouds folder exists
#If exists, delete all files in the folder
if os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    for file in os.listdir("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
        os.remove("{DATA_PATH}/pointclouds/{file}".format(DATA_PATH=DATA_PATH, file=file))
        
if not os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    os.makedirs("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH))

for i in range(len(images)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    confidence_mask = confidence_masks[i].detach().cpu().numpy()
    confidence_mask = confidence_mask.reshape(-1)
    
    masked_pointcloud = []
    masked_color = []

    for j in range(len(confidence_mask)):
        if confidence_mask[j]:
            masked_pointcloud.append(pointcloud[j])
            masked_color.append(color[j])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(masked_pointcloud)
    pcd.colors = o3d.utility.Vector3dVector(masked_color)
    o3d.io.write_point_cloud("{DATA_PATH}/pointclouds/pointcloud{i}.ply".format(DATA_PATH=DATA_PATH, i=i), pcd)

[Open3D WARNING] Write PLY failed: point cloud has 0 points.


[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.


In [17]:
#Create transform file

img = Image.open(images_array[0])
width, height = img.size
RESCALE_FACTOR  = width/512

transforms = {}
transforms["camera_model"] = "OPENCV"
if USE_COMMON_INTRINSICS:
    intrinsic_mean = intrinsics.mean(dim=0)
    transforms["fl_x"] = intrinsic_mean[0,0].item() * RESCALE_FACTOR
    transforms["fl_y"] = intrinsic_mean[1,1].item() * RESCALE_FACTOR
    transforms["w"] = width 
    transforms["h"] = height 
    transforms["cx"] = intrinsic_mean[0,2].item() * RESCALE_FACTOR
    transforms["cy"] = intrinsic_mean[1,2].item() * RESCALE_FACTOR

transforms["frames"] = []

OPENGL = np.array([[1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, -1, 0],
                    [0, 0, 0, 1]])

for i in range(len(poses)):
    if not((confidence_masks[i]==0).all()):
        frame = {}
        frame["file_path"] = "/".join(images_array[i].split("/")[-2:])
        #frame["transform_matrix"] = np.linalg.inv(poses[i].detach().cpu().numpy()).tolist()
        frame["transform_matrix"] = np.dot(poses[i].detach().cpu().numpy(),OPENGL).tolist()
        frame["mask_path"] = "/".join(masks_array[i].split("/")[-2:])
        transforms["frames"].append(frame)
        
        if not USE_COMMON_INTRINSICS:
            frame["fl_x"] = intrinsics[i,0,0].item() * RESCALE_FACTOR
            frame["fl_y"] = intrinsics[i,1,1].item() * RESCALE_FACTOR
            frame["cx"] = intrinsics[i,0,2].item() * RESCALE_FACTOR
            frame["cy"] = intrinsics[i,1,2].item() * RESCALE_FACTOR
            img = Image.open(images_array[i])
            width, height = img.size
            transforms["w"] = width 
            transforms["h"] = height 

#Save transform file
with open("{}/transforms.json".format(DATA_PATH), 'w') as f:
    json.dump(transforms, f, indent=4)

In [12]:
import roma

all_poses = torch.stack(list(scene.im_poses))
Q = all_poses[:,:4]
Q = torch.nn.functional.normalize(Q, p=2, dim=1)
T = signed_expm1(all_poses[:,4:7])
tf = roma.RigidUnitQuat(Q, T).normalize().to_homogeneous()

OPENGL = torch.tensor([[1, 0, 0, 0],
                       [0, -1, 0, 0],
                       [0, 0, -1, 0],
                       [0, 0, 0, 1]], dtype=torch.float32).to(device)

tf = torch.matmul(tf, OPENGL)

tf = roma.RigidUnitQuat(Q, T).normalize()
print(tf.linear[:,0]/tf.linear[:,3])

tensor([-3.9896e-02, -2.9239e-02, -3.1255e-02, -2.4163e-02, -9.6186e-03,
        -4.4208e-02, -9.6332e-02,  4.8475e-02,  4.1016e-02,  3.9322e-02,
         4.2743e-03,  1.2762e-03, -1.3266e-03,  1.7213e-03,  1.5212e-02,
         3.2159e-03, -1.0387e-02, -4.1131e-02, -4.2022e-02, -7.5971e-02,
        -2.0109e-02, -2.2821e-02, -2.6324e-02, -2.6963e-02, -2.6448e-02,
        -8.6911e-02, -5.0446e-02, -6.1260e-02, -6.3499e-02, -1.1514e-01,
        -2.5379e-02,  4.2996e-01,  1.3047e-01,  4.9400e-04,  2.7527e-02,
         1.4118e-02,  3.2864e-02, -1.2640e-02, -4.2873e-02,  1.0372e-02,
         6.0678e-03, -5.2305e-03,  2.0779e-03, -2.6433e-02, -9.1664e-02,
        -3.5783e-02, -1.1072e-03, -2.8934e-03,  1.0770e-02,  1.0044e-02,
        -1.9091e-03,  7.0740e-03,  1.1373e-02,  2.2223e-03, -2.6458e-02,
        -3.9399e-02, -6.8746e-01,  6.4164e-01,  7.0264e-02,  5.8222e-02,
         3.4702e-02,  1.9103e-02,  3.0426e-02, -3.3341e-02, -5.9395e-04,
         1.9871e-02,  3.3153e-02,  2.2435e-02, -9.2